In [3]:
import pandas as pd
import gzip
import pickle
import os

test_data = pd.read_csv(
        f"../files/input/test_data.csv.zip",
        index_col=False,
        compression="zip",
    )
train_data = pd.read_csv(
        f"../files/input/train_data.csv.zip",
        index_col=False,
        compression="zip",
    )

In [5]:

train_data.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,jazz,2016,7.40,8.500,15059,Petrol,Dealer,Automatic,0
1,i10,2013,4.00,4.600,30000,Petrol,Dealer,Manual,0
2,TVS Apache RTR 180,2011,0.50,0.826,6000,Petrol,Individual,Manual,0
3,eon,2016,3.15,4.430,15000,Petrol,Dealer,Manual,0
4,Royal Enfield Thunder 350,2013,1.25,1.500,15000,Petrol,Individual,Manual,0


In [6]:
train_data['Age']=2021-train_data['Year']

In [7]:
train_data.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,jazz,2016,7.40,8.500,15059,Petrol,Dealer,Automatic,0,5
1,i10,2013,4.00,4.600,30000,Petrol,Dealer,Manual,0,8
2,TVS Apache RTR 180,2011,0.50,0.826,6000,Petrol,Individual,Manual,0,10
3,eon,2016,3.15,4.430,15000,Petrol,Dealer,Manual,0,5
4,Royal Enfield Thunder 350,2013,1.25,1.500,15000,Petrol,Individual,Manual,0,8


In [8]:
def clean_data(df):
    #1. Calcular la columna Age
    df['Age']=2021-df['Year']
    # 2. Eliminar la columna 'Year' y 'Car_name' si existe
    if 'Car_Name' in df.columns:
        df = df.drop(columns=['Car_Name'])
    if 'Year' in df.columns:
        df = df.drop(columns=['Year'])
    # 3. Eliminar filas con valores NaN
    df = df.dropna()
    
    return df

train_data=clean_data(train_data)
test_data=clean_data(test_data)

In [9]:
x_train = train_data.drop(columns=['Selling_Price'])
y_train = train_data['Selling_Price']

x_test = test_data.drop(columns=['Selling_Price'])
y_test = test_data['Selling_Price']

In [10]:
x_train.columns

Index(['Present_Price', 'Driven_kms', 'Fuel_Type', 'Selling_type',
       'Transmission', 'Owner', 'Age'],
      dtype='object')

In [11]:
def make_pipeline(estimator):

    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
    from sklearn.feature_selection import SelectKBest, f_regression, SelectPercentile

# - Fuel_Type: Tipo de combustible.
# - Selling_type: Tipo de vendedor.
# - Transmission: Tipo de transmisión.
    cols=['Fuel_Type','Selling_type','Transmission']
    cols_num=[c for c in x_train.columns if c not in cols]
    print(cols_num)
    transformer = ColumnTransformer(
        transformers=[
            ("ohe", OneHotEncoder(dtype="int",handle_unknown="ignore"),cols),
            ("num", MinMaxScaler(), cols_num),
        ],
        remainder="passthrough",
    )
    selectkbest=SelectKBest(score_func=f_regression)
    
    pipeline = Pipeline(
        steps=[
            ("transformer", transformer),
            ("selectkbest", selectkbest),
            ("estimator", estimator),
        ],
        verbose=False
    )

    return pipeline


In [12]:
# Modelo estimador
from sklearn.linear_model import LinearRegression
rgr=LinearRegression()
rgr_estimator=make_pipeline(rgr)

['Present_Price', 'Driven_kms', 'Owner', 'Age']
